In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# import known packages
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import pickle
import gzip
import sys

sys.path.append('/Users/fradm/Desktop/fidelity-phase-tran')

# import adhoc packages
from qphaset.filters import SOBEL, SOBEL_X, bump_kernel, upsampling_base
from qphaset.fidelity import fidelity_laplacian, uhlmann_fidelity
from qs_mps.applications.Z2.utils import gstates_to_rdms_matrix_qs_mps, partial_rdm_n, single_rdm_n
from qs_mps.utils import get_cx, get_cy, tensor_shapes

path = "marcos"
if path == "pc":
    parent_path = f"C:/Users/HP/Desktop/projects/1_Z2"
    # parent_path = "G:/My Drive/projects/1_Z2"
    path_tensor = "D:/code/projects/1_Z2"
elif path == "mac":
    # parent_path = "/Users/fradm98/Google Drive/My Drive/projects/1_Z2"
    path_tensor = "/Users/fradm98/Desktop/projects/1_Z2"
    parent_path = path_tensor
elif path == "marcos":
    # parent_path = "/Users/fradm/Google Drive/My Drive/projects/1_Z2"
    path_tensor = "/Users/fradm/Desktop/projects/1_Z2"
    parent_path = path_tensor
else:
    raise SyntaxError("Path not valid. Choose among 'pc', 'mac', 'marcos'")


In [5]:
from qs_mps.mps_class import MPS

h_i, h_f, n = 0.8, 1.0, 41
gs = np.linspace(h_i,h_f,n)

l = 5
L = 30
chi = 64
R = 10
bc = "pbc"
model = "Z2_dual"
cx = get_cx(L, R)
cy = get_cy(l, bc)
precision = 3


sites = [L//2]
sites = [L//2]


if len(cx) == 0 and R == 0:
    sector = "vacuum_sector"
    cx = None
    cy = None
else:
    sector = f"{len(cx)}_particle(s)_sector"
    cx = cx
    cy = cy

rdms = []
for g in gs:
    print(f"rdm for g: {g:.3f}")
    lattice_mps = MPS(
        L=L, d=2**l, model=model, chi=chi, h=g, bc=bc
    )

    if sector != "vacuum_sector":
        lattice_mps.Z2.add_charges(cx, cy)
        lattice_mps.charges = lattice_mps.Z2.charges
        lattice_mps.Z2._define_sector()
    else:
        lattice_mps.Z2._define_sector()
    lattice_mps.load_sites(
        path=path_tensor, precision=precision, cx=cx, cy=cy
    )
    lattice_mps.mixed_canonical_form(site=sites[0], trunc_chi=False, trunc_tol=True, schmidt_tol=1e-15)
    # lattice_mps.check_canonical(site=sites[0])
    # tensor_shapes(lattice_mps.sites)
    # rdm = single_rdm_n(lattice_mps.sites, sites, L)
    rdm = lattice_mps.reduced_density_matrix_debug(sites=sites)
    rdms.append(rdm)

rdm for g: 0.800
Metadata: {'L': 30, 'bc': 'pbc', 'chi': 64, 'cx': array([10, 20]), 'cy': array([0, 0]), 'h': 0.8, 'l': 5, 'model': 'Z2_dual', 'sector': '2_particle(s)_sector'}
rdm for g: 0.805
Metadata: {'L': 30, 'bc': 'pbc', 'chi': 64, 'cx': array([10, 20]), 'cy': array([0, 0]), 'h': 0.805, 'l': 5, 'model': 'Z2_dual', 'sector': '2_particle(s)_sector'}
rdm for g: 0.810
Metadata: {'L': 30, 'bc': 'pbc', 'chi': 64, 'cx': array([10, 20]), 'cy': array([0, 0]), 'h': 0.81, 'l': 5, 'model': 'Z2_dual', 'sector': '2_particle(s)_sector'}
rdm for g: 0.815
Metadata: {'L': 30, 'bc': 'pbc', 'chi': 64, 'cx': array([10, 20]), 'cy': array([0, 0]), 'h': 0.8150000000000001, 'l': 5, 'model': 'Z2_dual', 'sector': '2_particle(s)_sector'}
rdm for g: 0.820
Metadata: {'L': 30, 'bc': 'pbc', 'chi': 64, 'cx': array([10, 20]), 'cy': array([0, 0]), 'h': 0.8200000000000001, 'l': 5, 'model': 'Z2_dual', 'sector': '2_particle(s)_sector'}
rdm for g: 0.825
Metadata: {'L': 30, 'bc': 'pbc', 'chi': 64, 'cx': array([10, 20])